In [1]:
%matplotlib inline
import nltk
import re
import sys
from nltk import word_tokenize
from tqdm import tqdm
print(sys.version)

3.6.5 |Anaconda, Inc.| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [2]:
kB = 1024
mB = 1024 * kB
collection = {}

In [3]:
def read_in_chunks(file_object, chunk_size=1024):
    while True:
        data = file_object.read(chunk_size)
        if not data:
            break
        yield data

In [4]:
import csv
news = []
with open('./data/news-week-aug24.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in tqdm(readCSV):
        news.append(row[3])

1398432it [00:05, 260335.70it/s]


In [5]:
from collections import Counter
wc = Counter()
for head in tqdm(news[1:]):
    tempt = word_tokenize(head)
    for word in tempt:
        wc[word] += 1

100%|██████████| 1398431/1398431 [02:52<00:00, 8116.58it/s]


In [ ]:
import pickle
with open('news.pickle', 'wb') as handle:
    pickle.dump(wc, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
count = 0
for key,value in wc.items():
    count += value
print(len(wc.keys())/count)

0.08158413897225916


In [9]:
len(wc.keys())

1186496

In [13]:
# vocab = set()
# wc = 0
# with open('./data/WestburyLab.Wikipedia.Corpus.txt') as file:
#     for chunk in tqdm(read_in_chunks(file, chunk_size=10*mB)):
#         tempt = word_tokenize(chunk)
#         wc += len(tempt)
#         vocab = vocab.union(set(tempt))
#     print(wc, len(vocab))

In [ ]:
# from collections import Counter
# all_words = []
# with open('./data/WestburyLab.Wikipedia.Corpus.txt') as file:
#     for chunk in tqdm(read_in_chunks(file, chunk_size=100*mB)):
#         tempt = word_tokenize(chunk)
#         all_words.append(tempt)

In [ ]:
ALL = []
for chunk in tqdm(all_words):
    for word in chunk:
        ALL.append(word)

In [75]:
WC = Counter(ALL)

In [10]:
import numpy as np
import matplotlib.pyplot as plt
labels, values = zip(*wc.items())

In [ ]:
plt.plot(labels,values)
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
plt.title("News Headline Word Distribution")
plt.xlabel("Vocabulary")
plt.ylabel("Count")
plt.savefig('newsHeadline')

In [ ]:
plt.savefig('newsHeadline')

In [54]:
with open("./wiki.txt", "w") as myFile:
    for word in vocab:
        myFile.write(word+'\n')

In [28]:
tokens = word_tokenize(collection[0][:100])

## Get a collection of 200MB    

In [27]:
kB = 1024
mB = 1024 * kB
collection = {}
with open('data/WestburyLab.Wikipedia.Corpus.txt','r') as file:
    index = 0
    for piece in tqdm(read_in_chunks(file, chunk_size=1*mB)):
        collection[index] = piece
        index += 1
        if index == 200:
            break

189it [00:00, 260.10it/s]


## Construct dictionary by regular sampling of size 1kB from first 100MB

In [4]:
dictionary = ""
for i in range(100):
    dictionary += collection[i][:1*kB]

In [5]:
len(dictionary)

102400

In [6]:
from difflib import SequenceMatcher

string1 = "apple pie available"
string2 = "come have some apple pies apple pies"

match = SequenceMatcher(None, string1, string2).find_longest_match(0, len(string1), 0, len(string2))

print(match)  # -> Match(a=0, b=15, size=9)
print(string1[match.a: match.a + match.size])  # -> apple pie
print(string2[match.b: match.b + match.size])  # -> apple pie

Match(a=0, b=15, size=9)
apple pie
apple pie


## Constrcut suffix array in $O(n\log^2{n})$

In [7]:
from math import ceil, log2
import numpy as np

# https://codereview.stackexchange.com/questions/87335/suffix-array-construction-in-on-log2-n
def sufar_np(txt):
    """
    This implements the algorithm of Vladu and Negruşeri; 
    see http://web.stanford.edu/class/cs97si/suffix-array.pdf
    """
    
    if not txt:
        return []
    txt += chr(0)

    equivalence = {t: i for i, t in enumerate(sorted(set(txt)))}
    cls = np.array([equivalence[t] for t in txt])
    ns = 2**np.arange(ceil(log2(len(txt))))

    for n in ns[:-1]:
        cls1 = np.roll(cls, -n)
        inds = np.lexsort((cls1, cls))
        result = np.logical_or(np.diff(cls[inds]), 
                               np.diff(cls1[inds]))

        cls[inds[0]] = 0
        cls[inds[1:]] = np.cumsum(result)

    cls1 = np.roll(cls, ns[-1])
    return np.lexsort((cls1, cls))[1:].tolist()

In [8]:
%time sufar_np('cabbaabba')

CPU times: user 1.08 ms, sys: 1.01 ms, total: 2.09 ms
Wall time: 7.19 ms


[8, 4, 5, 1, 7, 3, 6, 2, 0]

In [12]:
def factor(i, target, dictionary, sufix_d):
    
    i, lb = 0, 0
    rb = len(sufix_d) - 1
    found = False
    
    p = len(sufix_d)/2
    while lb <= rb and not found:
        mid = (lb+rb)//2
        t = target[:i+1]
        sub =  dictionary[sufix_d[mid]][:i+1]
        if t>sub:
            rb = mid
        elif t < sub:
            lb = mid
        # ??
        else:
            pass
        

In [19]:
[m.span() for m in re.finditer('anarchy', dictionary)]

[(157, 164)]

In [88]:
def longest_match(target, dictionary):
    curr = 0
    output = []
    target_len = len(target)
    while curr < target_len:
        i = 0
        go_on = True
        while go_on: 
            match = [m.span() for m in re.finditer(re.escape(target[:i+1]), dictionary)]
            longer_match = [m.span() for m in re.finditer(re.escape(target[:i+2]), dictionary)]
            if len(longer_match) == 0:
                if len(match) == 0:
                    curr += 1
                    output.append((target[i], 0))
                else:
                    length = match[0][1]-match[0][0]
                    curr += length
                    output.append((match[0][0],length))
                go_on = False
                target = target[i+1:]
            else:
                if i == len(target) - 1:
                    length = longer_match[0][1] - longer_match[0][0]
                    output.append((longer_match[0][0], length))
                    return output
                else:
                    i += 1
    return output

In [89]:
%time longest_match('anarchy ', dictionary)

CPU times: user 16.6 ms, sys: 1.78 ms, total: 18.3 ms
Wall time: 17.9 ms


[(157, 7), (21, 1)]

In [99]:
def foo():
    encode = []
    block_size = 1 * kB
    j = 0
    for i in range(100):
#         encode.append(longest_match(collection[i][j*block_size:(j+1)*block_size],dictionary
#         print(j)
#         j += 1
        encode.append(longest_match(collection[i],dictionary))

    return encode

In [ ]:
%time result = foo()

In [85]:
collection[1][1940:1943]

's ('

In [46]:
# from itertools import groupby
# from operator import itemgetter
# # https://stackoverflow.com/questions/13560037/effcient-way-to-find-longest-duplicate-string-for-python-from-programming-pearl/13693834#
# def longest_common_substring(text):
#     """Get the longest common substrings and their positions.
#     >>> longest_common_substring('banana')
#     {'ana': [1, 3]}
#     >>> text = "not so Agamemnon, who spoke fiercely to "
#     >>> sorted(longest_common_substring(text).items())
#     [(' s', [3, 21]), ('no', [0, 13]), ('o ', [5, 20, 38])]

#     This function can be easy modified for any criteria, e.g. for searching ten
#     longest non overlapping repeated substrings.
#     """
#     sa, rsa, lcp = suffix_array(text)
#     maxlen = max(lcp)
#     result = {}
#     for i in range(1, len(text)):
#         if lcp[i] == maxlen:
#             j1, j2, h = sa[i - 1], sa[i], lcp[i]
#             assert text[j1:j1 + h] == text[j2:j2 + h]
#             substring = text[j1:j1 + h]
#             if not substring in result:
#                 result[substring] = [j1]
#             result[substring].append(j2)
#     return dict((k, sorted(v)) for k, v in result.items())

# def suffix_array(text, _step=16):
#     """Analyze all common strings in the text.

#     Short substrings of the length _step a are first pre-sorted. The are the 
#     results repeatedly merged so that the garanteed number of compared
#     characters bytes is doubled in every iteration until all substrings are
#     sorted exactly.

#     Arguments:
#         text:  The text to be analyzed.
#         _step: Is only for optimization and testing. It is the optimal length
#                of substrings used for initial pre-sorting. The bigger value is
#                faster if there is enough memory. Memory requirements are
#                approximately (estimate for 32 bit Python 3.3):
#                    len(text) * (29 + (_size + 20 if _size > 2 else 0)) + 1MB

#     Return value:      (tuple)
#       (sa, rsa, lcp)
#         sa:  Suffix array                  for i in range(1, size):
#                assert text[sa[i-1]:] < text[sa[i]:]
#         rsa: Reverse suffix array          for i in range(size):
#                assert rsa[sa[i]] == i
#         lcp: Longest common prefix         for i in range(1, size):
#                assert text[sa[i-1]:sa[i-1]+lcp[i]] == text[sa[i]:sa[i]+lcp[i]]
#                if sa[i-1] + lcp[i] < len(text):
#                    assert text[sa[i-1] + lcp[i]] < text[sa[i] + lcp[i]]
#     >>> suffix_array(text='banana')
#     ([5, 3, 1, 0, 4, 2], [3, 2, 5, 1, 4, 0], [0, 1, 3, 0, 0, 2])

#     Explanation: 'a' < 'ana' < 'anana' < 'banana' < 'na' < 'nana'
#     The Longest Common String is 'ana': lcp[2] == 3 == len('ana')
#     It is between  tx[sa[1]:] == 'ana' < 'anana' == tx[sa[2]:]
#     """
#     tx = text
#     size = len(tx)
#     step = min(max(_step, 1), len(tx))
#     sa = list(range(len(tx)))
#     sa.sort(key=lambda i: tx[i:i + step])
#     grpstart = size * [False] + [True]  # a boolean map for iteration speedup.
#     # It helps to skip yet resolved values. The last value True is a sentinel.
#     rsa = size * [None]
#     stgrp, igrp = '', 0
#     for i, pos in enumerate(sa):
#         st = tx[pos:pos + step]
#         if st != stgrp:
#             grpstart[igrp] = (igrp < i - 1)
#             stgrp = st
#             igrp = i
#         rsa[pos] = igrp
#         sa[i] = pos
#     grpstart[igrp] = (igrp < size - 1 or size == 0)
#     while grpstart.index(True) < size:
#         # assert step <= size
#         nextgr = grpstart.index(True)
#         while nextgr < size:
#             igrp = nextgr
#             nextgr = grpstart.index(True, igrp + 1)
#             glist = []
#             for ig in range(igrp, nextgr):
#                 pos = sa[ig]
#                 if rsa[pos] != igrp:
#                     break
#                 newgr = rsa[pos + step] if pos + step < size else -1
#                 glist.append((newgr, pos))
#             glist.sort()
#             for ig, g in groupby(glist, key=itemgetter(0)):
#                 g = [x[1] for x in g]
#                 sa[igrp:igrp + len(g)] = g
#                 grpstart[igrp] = (len(g) > 1)
#                 for pos in g:
#                     rsa[pos] = igrp
#                 igrp += len(g)
#         step *= 2
#     del grpstart
#     # create LCP array
#     lcp = size * [None]
#     h = 0
#     for i in range(size):
#         if rsa[i] > 0:
#             j = sa[rsa[i] - 1]
#             while i != size - h and j != size - h and tx[i + h] == tx[j + h]:
#                 h += 1
#             lcp[rsa[i]] = h
#             if h > 0:
#                 h -= 1
#     if size > 0:
#         lcp[0] = 0
#     return sa, rsa, lcp